<a href="https://colab.research.google.com/github/Morgan-X1/church_web/blob/master/TraceGuard_Data_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:

df = pd.read_csv('/content/HI-Small_Trans.csv')

# Display the first 5 rows to see the structure
df.head()

,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
0,2022/09/01 00:20,10.0,8000EBD30,10.0,8000EBD30,3697.34,US Dollar,3697.34,US Dollar,Reinvestment,0.0
1,2022/09/01 00:20,3208.0,8000F4580,1.0,8000F5340,0.01,US Dollar,0.01,US Dollar,Cheque,0.0
2,2022/09/01 00:00,3209.0,8000F4670,3209.0,8000F4670,14675.57,US Dollar,14675.57,US Dollar,Reinvestment,0.0
3,2022/09/01 00:02,12.0,8000F5030,12.0,8000F5030,2806.97,US Dollar,2806.97,US Dollar,Reinvestment,0.0
4,2022/09/01 00:06,10.0,8000F5200,10.0,8000F5200,36682.97,US Dollar,36682.97,US Dollar,Reinvestment,0.0


In [3]:
# Create a dictionary for exchange rates (2026 estimates)
# Add any other currencies you found in your unique check
exchange_rates = {
    'US Dollar': 1.0,
    'Euro': 1.10,
    'British Pound': 1.25,
    'Swiss Franc': 1.15,
    'Yen': 0.0067
}

# Function to convert amounts to USD
def normalize_to_usd(row):
    currency = row['Receiving Currency']
    amount = row['Amount Received']
    rate = exchange_rates.get(currency, 1.0) # Defaults to 1.0 if currency not found
    return amount * rate

# Apply normalization
df['Amount_USD'] = df.apply(normalize_to_usd, axis=1)

In [10]:
import pandas as pd

# 1. Clean Timestamps (Using errors='coerce' to handle unrecognized formats)
df['Timestamp'] = pd.to_datetime(df['Timestamp'], errors='coerce')

# 2. Drop rows with invalid Timestamps (NaT) to prevent rolling errors
df = df.dropna(subset=['Timestamp'])

# 3. Sort for accurate time-based rolling
df = df.sort_values(by=['Account', 'Timestamp'])

# 4. Use the Index-Based Approach (Solves the KeyError and Window issues)
# We set Timestamp as the index to allow the '2D' window to function as a clock
df = df.set_index('Timestamp')

# 5. Group by Account and calculate the 2-day rolling count
# We count 'From Bank' because it's a stable, non-null column for every transaction
df['Tx_Velocity_24h'] = df.groupby('Account')['From Bank'].rolling('2D').count().reset_index(level=0, drop=True)

# 6. Reset index to return 'Timestamp' to a regular column for your AI model
df = df.reset_index()

In [11]:
# Check how many '1's (criminals) were caught by your Structuring Flag
correlation_check = df.groupby('Is Laundering')['Is_Structuring_Risk'].mean()
print("Percentage of 'Is_Structuring_Risk' in each category:")
print(correlation_check)

# See if 'Launderers' have a higher average Velocity than normal accounts
velocity_check = df.groupby('Is Laundering')['Tx_Velocity_24h'].mean()
print("\nAverage 24h Velocity:")
print(velocity_check)

KeyError: 'Column not found: Is_Structuring_Risk'

In [12]:

df['Is_Structuring_Risk'] = ((df['Amount_USD'] >= 9000) & (df['Amount_USD'] < 10000)).astype(int)


print("Current Columns:", df.columns.tolist())


correlation_check = df.groupby('Is Laundering')['Is_Structuring_Risk'].mean()
print("\nPercentage of Structuring Risk per Category:")
print(correlation_check)

Current Columns: ['Timestamp', 'From Bank', 'Account', 'To Bank', 'Account.1', 'Amount Received', 'Receiving Currency', 'Amount Paid', 'Payment Currency', 'Payment Format', 'Is Laundering', 'Amount_USD', 'Tx_Velocity_24h', 'Is_Structuring_Risk']

Percentage of Structuring Risk per Category:
Is Laundering
0.0    0.010376
1.0    0.029412
Name: Is_Structuring_Risk, dtype: float64
